In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col
import pyspark.pandas as ps
spark = SparkSession.builder.master("local[2]").appName("Spark").getOrCreate();

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/08 15:18:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "admin")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "password")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

In [27]:
rootPath = f"s3a://learning/Job_Posting"
filePrefix = f".csv"
posting_data = spark.read.option("header",True).csv("{rootPath}/postings{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));

#Companies Input Data
companies_Data = spark.read.option("header",True).csv("{rootPath}/companies/companies{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));
company_Industries_Data = spark.read.option("header",True).csv("{rootPath}/companies/company_industries{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));
company_Specialities_Data = spark.read.option("header",True).csv("{rootPath}/companies/company_specialities{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));
employee_Counts_Data = spark.read.option("header",True).csv("{rootPath}/companies/employee_counts{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));

#Jobs Input Data
benefits_Data = spark.read.option("header",True).csv("{rootPath}/jobs/benefits{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));
job_Industries_Data = spark.read.option("header",True).csv("{rootPath}/jobs/job_industries{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));
job_Skills_Data = spark.read.option("header",True).csv("{rootPath}/jobs/job_skills{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));
salaries_Data = spark.read.option("header",True).csv("{rootPath}/jobs/salaries{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));

#mapping between data 
industries_Mapping_Data = spark.read.option("header",True).csv("{rootPath}/mappings/industries{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));
skills_Mapping_Data = spark.read.option("header",True).csv("{rootPath}/mappings/skills{filePrefix}".format(rootPath = rootPath,filePrefix = filePrefix));

In [28]:
companies_Data.columns

['company_id',
 'name',
 'description',
 'company_size',
 'state',
 'country',
 'city',
 'zip_code',
 'address',
 'url']

In [29]:
#Cast the primary key properly as there might issue with data types
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
posting_data = posting_data.withColumn('company_id', col('company_id').cast('integer')) \
                               .withColumn('job_id', col('job_id').cast('integer'))

In [37]:
#Model the data 
modeled_Posting_Data = posting_data.join(companies_Data, posting_data.company_id == companies_Data.company_id, "leftouter") \
                                    .select(posting_data["*"],companies_Data["description"].alias("company_description"),companies_Data["company_size"],companies_Data["state"])

In [38]:
modeled_Posting_Data.filter("state != null or state != '' ").pandas_api().take([10])

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,min_salary,formatted_work_type,applies,original_listed_time,remote_allowed,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,company_description,company_size,state
10,NaN,White Bottle Inc.,Sanitation Supervisor,"Location: Stanton, CaliforniaStatus and Classi...",27.0,HOURLY,"Stanton, CA",82540802,9.0,None,25.0,Full-time,None,1713460499000.0,None,https://www.linkedin.com/jobs/view/3363439517/...,None,ComplexOnsiteApply,1716052499000.0,None,None,None,1713460499000.0,None,0,FULL_TIME,USD,BASE_SALARY,54080.0,90680,06059,None,2,California
